In [1]:
import os
import sys
import base64
import requests
from typing import Union
from requests.auth import HTTPBasicAuth
from requests.exceptions import HTTPError

In [2]:
project_path = os.path.abspath('..')
if project_path not in sys.path:
    sys.path.append(project_path)
sys.path

['C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner\\src',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3',
 '',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner']

In [3]:
from config.config import config

In [4]:
auth = HTTPBasicAuth(config['user_1C'], config['password_1C'])

### test functions

In [5]:
deal_numbers_list = requests.get(r'http://10.10.0.10:81/ca/hs/interaction/InvoicesByDate/01-10-2024/10-10-2024', auth=auth).json()
len(deal_numbers_list)

587

In [6]:
deal_numbers_list[0:5]

['0С000000000000143272',
 '0С000000000000143273',
 '0С000000000000143282',
 '0С000000000000143283',
 '0С000000000000143329']

In [37]:
deal = deal_numbers_list[0]
print(deal)

res = requests.get(fr'http://10.10.0.10:81/ca/hs/interaction/InvoiceDetailsByNumber/{deal}', auth=auth).json()

0С000000000000143272


In [38]:
list(res)

['СтрокиСчета', 'ФайлыСчета']

In [43]:
res['СтрокиСчета'][0:2]

[{'Услуга': 'Организация автотранспортной перевозки',
  'УслугаКод': '000000190',
  'ТранспортнаяПозиция': 'SEGU9857890',
  'Количество': 1,
  'Цена': 12000},
 {'Услуга': 'Организация автотранспортной перевозки',
  'УслугаКод': '000000190',
  'ТранспортнаяПозиция': 'FOHU2060420',
  'Количество': 1,
  'Цена': 12000}]

In [44]:
res['ФайлыСчета'][0:2]

['\\\\10.10.0.3\\docs\\Baltimpex\\Invoice\\TR\\Import\\НОВ-119410\\Счет поставщика\\сч 81 от 01.10 Гаспарян ИП 01.10.2024.pdf',
 '\\\\10.10.0.3\\docs\\Baltimpex\\Invoice\\TR\\Import\\НОВ-119438\\Счет поставщика\\сч 81 от 01.10 Гаспарян ИП 01.10.2024.pdf']

In [55]:
import hashlib


def calculate_hash(file_path):
    # Инициализация хэш-объекта MD5
    md5_hash = hashlib.md5()

    # Открываем файл в бинарном режиме для чтения
    with open(file_path, "rb") as f:
        # Чтение файла блоками по 4096 байт (можно настроить)
        for byte_block in iter(lambda: f.read(4096), b""):
            md5_hash.update(byte_block)

    # Возвращаем хэш-сумму в виде шестнадцатеричной строки
    return md5_hash.hexdigest()


def is_single_hash(lst: list[str]) -> str | None:
    if not list:
        return None
        
    res = [calculate_hash(file) for file in lst]
        
    if len(set(res)) == 1:
        return lst[0]

In [56]:
is_single_hash(res['ФайлыСчета'])

'\\\\10.10.0.3\\docs\\Baltimpex\\Invoice\\TR\\Import\\НОВ-119410\\Счет поставщика\\сч 81 от 01.10 Гаспарян ИП 01.10.2024.pdf'

In [68]:
import re
import json


file = os.path.join(config['BASE_DIR'], 'config', 'freq_services.json')
with open(file, 'r', encoding='utf-8') as f:
    freq_services = json.load(f)


def remove_special_characters(text):
    response = re.sub(r'[^\w\s]', ' ', text.lower())
    return re.sub(r'(\s{2,}|\n)', ' ', response)
    
freq_services = [remove_special_characters(service) for service in freq_services]

In [69]:
def is_all_services_freq(lst: list, freq_services: list) -> bool:
    """
    lst: result['СтрокиСчета'] from result = requests.get(fr'http://.../InvoiceDetailsByNumber/{deal}', auth=auth).json()
    freq_services: list of frequently used services (cleared)
    """
    services = [d['Услуга'] for d in lst]
    clear_services = [remove_special_characters(service) for service in services]
    
    if set(clear_services).issubset(freq_services):
        return True
    else:
        return False    

In [67]:
a.issubset(b)

True

In [70]:
freq_services

['проведение экспертной оценки в целях фитосанитарной экспертизы и установления фитосанитарного состояния подкарантинной продукции',
 'экспертиза ед',
 'тэо по организации погрузочно разгрузочных работ',
 'разрешение на использование контейнеров на территории рф',
 'документационные услуги',
 'оформление заключения о карантинном фитосанитарном состоянии',
 'доставка проб в лабораторию',
 'оформление протокола испытаний',
 'тэо по организации хранения контейнеров на терминале',
 'страховая премия',
 'тэо по организации перемещения контейнера в рамках досмотровых операций',
 'сюрвейерская инспекция груза',
 'лабораторные исследования товара',
 'перевалка назем транспортное средство контейнер исо 20 40 груженый импорт ',
 'услуга платежного агента по сбору и перечислению сумм причитающихся принципалу',
 'дегазация подкарантинной продукции в морском контейнере 40 футов',
 'оформление результатов',
 'сбор за оформление и выдачу грузовых документов',
 'оформление пропусков на грузовые а м',
